In [1]:
%run ./../template.ipynb

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


template load succesfull!


In [2]:
path = kagglehub.dataset_download("rohanrao/air-quality-data-in-india")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\2814\.cache\kagglehub\datasets\rohanrao\air-quality-data-in-india\versions\12


In [ ]:
data_path = Path(config['ml']['data_folder'])
shutil.move(path, data_path)

In [4]:
train_df = pd.read_csv(data_path / "12/city_day.csv")
val_df = pd.read_csv(data_path / "12/city_day.csv")

In [8]:
train_df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [5]:
print(val_df.shape)

(29531, 16)


In [ ]:
# Пропущенные поля
print(train_df.isnull().sum())
# Убираем колонки 
train_df.drop(columns=["City", "Date"], inplace=True)
# Заполнение NaN (например, медианой или средним) - надо ли нам это?
# train_df.head().fillna(train_df.head().median(), inplace=True) 

In [13]:
# Здесь мы фильтруем данные для города "Ahmedabad" и выбираем столбцы "Date", "PM2.5", "CO", "O3"
city_data = train_df[train_df["City"] == "Ahmedabad"][["Date", "PM2.5", "CO", "O3"]]

# Теперь заполним пропуски (NaN) в выбранных столбцах медианой этих столбцов
city_data[["PM2.5", "CO", "O3"]] = city_data[["PM2.5", "CO", "O3"]].fillna(city_data[["PM2.5", "CO", "O3"]].median())
print(city_data)

            Date  PM2.5     CO      O3
0     2015-01-01  58.37   0.92  133.36
1     2015-01-02  58.37   0.97   34.06
2     2015-01-03  58.37  17.40   30.70
3     2015-01-04  58.37   1.70   36.08
4     2015-01-05  58.37  22.10   39.31
...          ...    ...    ...     ...
2004  2020-06-27  62.12   0.49   68.05
2005  2020-06-28  31.57   0.52   26.34
2006  2020-06-29  29.75   0.67   34.99
2007  2020-06-30  40.02   0.73   41.64
2008  2020-07-01  37.63   0.28    9.69

[2009 rows x 4 columns]


In [2]:
from transformers import TimeSeriesTransformerForPrediction, TimeSeriesTransformerConfig
model_name = "kleopatra102/air_pollution"
config = TimeSeriesTransformerConfig.from_pretrained(model_name)
model = TimeSeriesTransformerForPrediction.from_pretrained(model_name)
# print(model.config)

In [ ]:

X_test = np.random.randn(100, 10, 270) 
past_time_features = np.random.randn(100, 10, 270) 
past_observed_mask = np.ones((100, 10)) 
static_categorical_features = torch.zeros((100, 5))  
static_real_features = torch.zeros((100, 3)) 

X_test = torch.tensor(X_test, dtype=torch.float32)
past_time_features = torch.tensor(past_time_features, dtype=torch.float32)
past_observed_mask = torch.tensor(past_observed_mask, dtype=torch.float32)


model.eval() 
with torch.no_grad():
    predictions = model(
        X_test, 
        past_time_features, 
        past_observed_mask,
        static_categorical_features=static_categorical_features, 
        static_real_features=static_real_features
    )
print(f"Shape of predictions: {predictions.shape}")

(100, 10, 270)
(100, 10, 270)
(100, 10)


RuntimeError: The size of tensor a (270) must match the size of tensor b (10) at non-singleton dimension 2

In [23]:
help(model.forward)

Help on method forward in module transformers.models.time_series_transformer.modeling_time_series_transformer:

forward(
    past_values: torch.Tensor,
    past_time_features: torch.Tensor,
    past_observed_mask: torch.Tensor,
    static_categorical_features: Optional[torch.Tensor] = None,
    static_real_features: Optional[torch.Tensor] = None,
    future_values: Optional[torch.Tensor] = None,
    future_time_features: Optional[torch.Tensor] = None,
    future_observed_mask: Optional[torch.Tensor] = None,
    decoder_attention_mask: Optional[torch.LongTensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    decoder_head_mask: Optional[torch.Tensor] = None,
    cross_attn_head_mask: Optional[torch.Tensor] = None,
    encoder_outputs: Optional[List[torch.FloatTensor]] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    output_hidden_states: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    use_cache: Optional[bool] = None,
    retu

In [24]:
print("d_model:", config.d_model) 
print(model.forward.__doc__)

d_model: 32
   The [`TimeSeriesTransformerForPrediction`] forward method, overrides the `__call__` special method.

    <Tip>

    Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
    instance afterwards instead of this since the former takes care of running the pre and post processing steps while
    the latter silently ignores them.

    </Tip>

    Args:
        past_values (`torch.FloatTensor` of shape `(batch_size, sequence_length)` or `(batch_size, sequence_length, input_size)`):
            Past values of the time series, that serve as context in order to predict the future. The sequence size of
            this tensor must be larger than the `context_length` of the model, since the model will use the larger size
            to construct lag features, i.e. additional values from the past which are added in order to serve as "extra
            context".

            The `sequence_length` here is equal to `config.context

In [3]:
time_steps = 100 
features = 5 

# Генерация случайных данных для временного ряда
data = np.random.rand(time_steps, features)  

df = pd.DataFrame(data, columns=[f"feature_{i}" for i in range(features)])
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4
0,0.948689,0.023647,0.373192,0.821342,0.633220
1,0.661424,0.825077,0.724590,0.450877,0.162731
2,0.101875,0.451638,0.786120,0.745441,0.410758
3,0.254375,0.571630,0.900094,0.696035,0.417817
4,0.809517,0.327158,0.782221,0.019583,0.990781


In [8]:
time_steps = 100  # Количество временных шагов
expected_features = 48  # Ожидаемое количество признаков
inputs = torch.randn(1, time_steps, expected_features)  # (batch_size, time_steps, expected_features)
past_time_features = torch.randn(1, time_steps, expected_features)  # Временные признаки
past_observed_mask = torch.ones(1, time_steps) # Маска наблюдаемых данных

In [9]:
model.eval()  # Переводим модель в режим инференса

with torch.no_grad():  # Отключаем вычисление градиентов
        outputs = model(inputs, 
                     past_time_features=past_time_features, 
                     past_observed_mask=past_observed_mask)
# В outputs будет содержаться результат, например, прогноз
print(outputs)

RuntimeError: expand(torch.FloatTensor{[1, 1, 2, 48]}, size=[-1, 0, -1]): the number of sizes provided (3) must be greater or equal to the number of dimensions in the tensor (4)